To pair satellite images with PM and Met, this script requires:
- CSV with location names and coordinates 
- Folder with all locations' PM in seperate .xls files. .xls files should be named after location, matching in the othe CSV with location names and coordinates.


In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals


# Common imports
import numpy as np
import pandas as pd
import os
# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import rasterio
from rasterio.merge import merge
from rasterio.plot import show

import json
from shapely.geometry import shape
from shapely.ops import transform as stransform
from rasterio.mask import mask
from affine import Affine
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pyproj
import pickle as pkl
import geojson
import cv2

from rasterio.warp import transform as rtransform
from rasterio.transform import rowcol

import PIL
from PIL import Image, ImageEnhance
import sys
# from skimage import io

os.getcwd()

'/Users/sarah/Satellite_Image_Processing'

We need the coordinates of our sensors

In [2]:
# #coordinates
my_coordinates = pd.read_excel('../Documents/MultiCityData/IndiaData/ALL_sensors.xlsx')
my_coordinates = my_coordinates.iloc[76:77]
my_coordinates

,SITE,latitude,longitude
76,Lahore,31.559696,74.336334


Below is where we create the dictionary that stores the dates and PM. A couple notes:
* This dictionary is indexed by date.
* You must change the paths in the block to your specific paths for where your PM spreadsheets reside. Each spreadsheet should contain only two columns, the date and PM of your data and the headers should be `DATE` and `PM`

In [3]:
PM25_dict = {}
for file in sorted(listdir('../Documents/MultiCityData/Lahore_PM_Spreadsheets/')):  #[1:] because MacOS has a weird first output
    if '.xls' in file and file != '.DS_Store':
#         print(file)

        site_name = file.split('.xls')[0]
        print(site_name)
        site_coordinate = (my_coordinates[my_coordinates.SITE == site_name].values[0,1],\
                            my_coordinates[my_coordinates.SITE == site_name].values[0,2])
        print(site_coordinate)
        try:
            df = pd.read_excel(join('../Documents/MultiCityData/Lahore_PM_Spreadsheets/', file))
            df['DATE'] = pd.to_datetime(df['DATE'],dayfirst=False)
            df = df.set_index(pd.DatetimeIndex(df['DATE']))
            df = df[df['PM']!= 'None']
            df_PM25 = df['PM'].to_frame().dropna()
            
#             print(df)
            if len(df_PM25)>0:
                PM25_dict[site_name] = {'PM':df_PM25, 'coordinate':site_coordinate}
       
        except Exception as e:
            print('error:',e.args)

Lahore
(31.5596963245208, 74.3363335065986)


In [4]:
df_PM25

,PM
DATE,
2019-05-09,166.41
2019-05-10,95.82
2019-05-11,150.64
2019-05-12,120.98
2019-05-13,130.71
...,...
2021-12-26,511.75
2021-12-27,360.78
2021-12-28,169.89


The below block pairs the image and splices the image ID to retrieve `Date`, `Time_stamp`, `Station_index`, and `Sat_ID`. Notes:
* images should be cropped to your liking. This is the input we feed into an ML network.
* There is an additional filter for images that are too blocked out or too opaque from cloud/reflection.

In [5]:
def Imagery_matcher(final_image_path):
    try:
#         print('site loading')
        im = np.moveaxis(rasterio.open(final_image_path).read(), 0, 2)
        img_arr = np.array(im)
#         print('read in working')
    except Exception as e:
        print(e)
        return None
    
    black_space = np.mean(img_arr[:,:,:]/255)


    if black_space > .91 or black_space <= 0.4:
#         print(black_space)
#         plt.imshow(im)
#         plt.show()
        return None
    else:
        
        station_index = final_image_path.split('/')[3]
        time_index = final_image_path.split('/')[-1].split('_')[1]
        time_index = time_index[0:2]+':'+time_index[2:4]+':' +time_index[4:]
        date_index = final_image_path.split('/')[-1].split('_')[0]
        date_index = date_index[0:4] + '-' + date_index[4:6] + '-' + date_index[6:]
        id_index = final_image_path.split('/')[-1].split('_')[2]
#         print(station_index, time_index, date_index, id_index)
        
    try:
        matching_PM25 = PM25_dict[station_index]['PM'].loc[date_index, 'PM'].squeeze()
        print(type(matching_PM25))
        if isinstance(matching_PM25, pd.Series):
            print('uh oh')
            return None
#         print('matched pm working')
        

        
    except Exception as e:
        print('error:', e)
        return None

    return({'Image':im, 'PM':matching_PM25, 'Date':date_index, 'Time_stamp':time_index,
            'Station_index': station_index, 'Sat_ID':id_index})

In [6]:
def Image_loader(image_folder_path):
    print(image_folder_path)
    Matching_data_for_a_single_station = [Imagery_matcher(join(image_folder_path, f)) for f in sorted(listdir(image_folder_path)) \
                            if isfile(join(image_folder_path, f)) and '.DS_Store' not in str(f)\
                                         and Imagery_matcher(join(image_folder_path, f)) is not None]
    return Matching_data_for_a_single_station

In [7]:
AQM_data_for_model = [Image_loader('../Desktop/' + site) for site in my_coordinates.SITE]

../Desktop/Lahore
error: '20170102_045808_0e14_3B_Visual_clip.tif'
error: '20170120_045833_0e2f_3B_Visual_clip.tif'
error: '20170208_045846_0e26_3B_Visual_clip.tif'
error: '20170212_050003_0e3a_3B_Visual_clip.tif'
error: '20170213_045810_0e26_3B_Visual_clip.tif'
error: '20170219_045913_0e0f_3B_Visual_clip.tif'
error: '20170222_024752_0c19_3B_Visual_clip.tif'
error: '20170226_045905_0e30_3B_Visual_clip.tif'
error: '20170227_084943_0c19_3B_Visual_clip.tif'
error: '20170302_050002_0e14_3B_Visual_clip.tif'
error: '20170313_032104_0c37_3B_Visual_clip.tif'
error: '20170321_050000_0e20_3B_Visual_clip.tif'
error: '20170325_050036_0e0f_3B_Visual_clip.tif'
error: '20170327_045256_1031_3B_Visual_clip.tif'
error: '20170328_045245_0f18_3B_Visual_clip.tif'
error: '20170402_045317_1003_3B_Visual_clip.tif'
error: '20170406_100522_0c54_3B_Visual_clip.tif'
error: '20170412_045310_1024_3B_Visual_clip.tif'
error: '20170414_045343_100f_3B_Visual_clip.tif'
error: '20170415_055000_0d05_3B_Visual_clip.tif'
er

/Users/sarah/opt/anaconda3/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


error: '20170614_045633_100a_3B_Visual_clip.tif'
error: '20170616_045639_0f15_3B_Visual_clip.tif'
error: '20170618_045215_0c0b_3B_Visual_clip.tif'
error: '20170619_045629_1042_3B_Visual_clip.tif'
error: '20170622_045640_102d_3B_Visual_clip.tif'
error: '20170623_045706_1036_3B_Visual_clip.tif'
error: '20170624_045704_102e_3B_Visual_clip.tif'
error: '20170625_045746_0f51_3B_Visual_clip.tif'
error: '20170626_045627_0f35_3B_Visual_clip.tif'
error: '20170627_045644_1030_3B_Visual_clip.tif'
error: '20170701_045536_102a_3B_Visual_clip.tif'
error: '20170703_045926_1044_3B_Visual_clip.tif'
error: '20170707_045829_100a_3B_Visual_clip.tif'
error: '20170708_045637_1004_3B_Visual_clip.tif'
error: '20170709_034728_0c43_3B_Visual_clip.tif'
error: '20170710_030337_0c13_3B_Visual_clip.tif'
error: '20170714_023835_0c56_3B_Visual_clip.tif'
error: '20170715_010350_0c81_3B_Visual_clip.tif'
error: '20170721_045825_1031_3B_Visual_clip.tif'
error: '20170723_045822_0f12_3B_Visual_clip.tif'
error: '20170725_125

error: '20180624_051159_103e_3B_Visual_clip.tif'
error: '20180625_051203_102e_3B_Visual_clip.tif'
error: '20180626_052116_0f49_3B_Visual_clip.tif'
error: '20180702_051141_1029_3B_Visual_clip.tif'
error: '20180706_052040_104d_3B_Visual_clip.tif'
error: '20180707_051252_1015_3B_Visual_clip.tif'
error: '20180708_051234_1004_3B_Visual_clip.tif'
error: '20180709_051315_1033_3B_Visual_clip.tif'
error: '20180710_051142_1009_3B_Visual_clip.tif'
error: '20180714_051215_0e0f_3B_Visual_clip.tif'
error: '20180715_051921_0f2a_3B_Visual_clip.tif'
error: '20180716_051254_1027_3B_Visual_clip.tif'
error: '20180718_051152_1035_3B_Visual_clip.tif'
error: '20180719_051217_1029_3B_Visual_clip.tif'
error: '20180723_051317_101f_3B_Visual_clip.tif'
error: '20180725_051251_1004_3B_Visual_clip.tif'
error: '20180726_051738_104a_3B_Visual_clip.tif'
error: '20180730_051301_103d_3B_Visual_clip.tif'
error: '20180731_051159_0e0f_3B_Visual_clip.tif'
error: '20180803_051428_0f43_3B_Visual_clip.tif'
error: '20180804_051

error: '20190805_052345_100c_3B_Visual_clip.tif'
error: '20190806_053741_1060_3B_Visual_clip.tif'
error: '20190811_052420_1004_3B_Visual_clip.tif'
error: '20190815_041852_0f33_3B_Visual_clip.tif'
error: '20190816_052152_103b_3B_Visual_clip.tif'
error: '20190818_052151_1024_3B_Visual_clip.tif'
error: '20190821_052355_0f3f_3B_Visual_clip.tif'
error: '20190822_041936_0f1a_3B_Visual_clip.tif'
error: '20190824_052234_0f28_3B_Visual_clip.tif'
error: '20190825_052238_1032_3B_Visual_clip.tif'
error: '20190826_052338_1025_3B_Visual_clip.tif'
error: '20190827_052427_1044_3B_Visual_clip.tif'
error: '20190829_041541_104a_3B_Visual_clip.tif'
error: '20190830_052122_1001_3B_Visual_clip.tif'
error: '20190901_054301_105c_3B_Visual_clip.tif'
error: '20190903_041649_1048_3B_Visual_clip.tif'
error: '20190904_052222_1027_3B_Visual_clip.tif'
error: '20190906_052145_1014_3B_Visual_clip.tif'
error: '20190907_052120_1003_3B_Visual_clip.tif'
error: '20190908_041626_1020_3B_Visual_clip.tif'
error: '20190909_041

error: '20201002_031120_1050_3B_Visual_clip.tif'
error: '20201003_052506_103c_3B_Visual_clip.tif'
error: '20201004_050013_222b_3B_Visual_clip.tif'
error: '20201005_031257_0f21_3B_Visual_clip.tif'
error: '20201006_031147_0f4d_3B_Visual_clip.tif'
error: '20201007_045801_220b_3B_Visual_clip.tif'
error: '20201011_045846_222c_3B_Visual_clip.tif'
error: '20201012_031120_1053_3B_Visual_clip.tif'
error: '20201013_045954_222b_3B_Visual_clip.tif'
error: '20201014_052541_1034_3B_Visual_clip.tif'
error: '20201015_030920_0f36_3B_Visual_clip.tif'
error: '20201016_052712_1038_3B_Visual_clip.tif'
error: '20201017_054928_2307_3B_Visual_clip.tif'
error: '20201018_050326_2278_3B_Visual_clip.tif'
error: '20201019_053205_1062_3B_Visual_clip.tif'
error: '20201020_031106_0f3c_3B_Visual_clip.tif'
error: '20201021_054037_1105_3B_Visual_clip.tif'
error: '20201022_030807_100d_3B_Visual_clip.tif'
error: '20201023_052620_1008_3B_Visual_clip.tif'
error: '20201024_030921_1052_3B_Visual_clip.tif'
error: '20201025_054

error: '20210826_021842_1053_3B_Visual_clip.tif'
error: '20210827_045227_2448_3B_Visual_clip.tif'
error: '20210828_050528_2251_3B_Visual_clip.tif'
error: '20210829_021742_0f3c_3B_Visual_clip.tif'
error: '20210830_045608_2455_3B_Visual_clip.tif'
error: '20210831_045631_0e20_3B_Visual_clip.tif'
error: '20210904_045340_2436_3B_Visual_clip.tif'
error: '20210909_021439_0f44_3B_Visual_clip.tif'
error: '20210913_045437_2440_3B_Visual_clip.tif'
error: '20210914_045751_2421_3B_Visual_clip.tif'
error: '20210915_054251_240a_3B_Visual_clip.tif'
error: '20210916_054156_241c_3B_Visual_clip.tif'
error: '20210922_051908_1011_3B_Visual_clip.tif'
error: '20210924_052132_1009_3B_Visual_clip.tif'
error: '20210925_053022_1026_3B_Visual_clip.tif'
error: '20210926_045344_241d_3B_Visual_clip.tif'
error: '20210927_045726_241e_3B_Visual_clip.tif'
error: '20210929_045255_2448_3B_Visual_clip.tif'
error: '20210930_045451_2212_3B_Visual_clip.tif'
error: '20211001_045557_220b_3B_Visual_clip.tif'
error: '20211002_045

In [8]:
len(AQM_data_for_model)

1

In [9]:
AQM_data_for_model[0]

[]

In [1]:
# for j in range(0,9):
#     for i in range(len(AQM_data_for_model[j])):
#         print(AQM_data_for_model[j][i]['PM'])
# #     h = len(AQM_data_for_model[i])
# #     print(np.sum(h))

In [2]:
# plt.imshow(AQM_data_for_model[6][300]['Image'])
# print(AQM_data_for_model[6][300])

In [22]:
with open("../Desktop/Jaipar.pkl", "wb") as fp:
    pkl.dump(AQM_data_for_model, fp)